In [41]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
#import prediction
#from utils import load_data,get_data_day
import logging
logging.basicConfig(level=logging.INFO)
%matplotlib inline
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
import utils
from keras.models import load_model
from keras import backend as K
from utils import load_data
from utils import get_data_day
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
ddd=load_data()

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


In [7]:
sites=ddd['sites']

In [9]:
sites['idPolair']=pd.to_numeric(sites['idPolair'])
sites=sites.set_index('idPolair')

In [35]:
sites

,coord_x_l93,coord_y_l93
idPolair,,
15013,91.466844,644.596850
15017,91.097088,645.816550
15018,90.269900,647.672600
15031,97.331581,643.921300
15038,91.620200,645.751600
15039,91.241138,645.465050
15043,91.491600,645.510200
15045,91.067863,644.343650
15046,91.365000,645.717500


In [11]:
sites=sites.iloc[:,1:3]

In [13]:
sites=sites/10000

In [15]:
sites.head()

,coord_x_l93,coord_y_l93
idPolair,,
15013,91.466844,644.59685
15017,91.097088,645.81655
15018,90.269900,647.67260
15031,97.331581,643.92130
15038,91.620200,645.75160


In [2]:
data=pd.read_csv('fillna/final.csv')
data['date']=pd.to_datetime(data['date'])
data=data.set_index(['idPolair','date'])

In [16]:
newData=pd.merge(sites,data.reset_index().set_index('idPolair'),left_index=True,right_index=True,how='left')

In [20]:
newData=newData.reset_index().set_index(['idPolair','date'])

In [22]:
newData.corr()

,coord_x_l93,coord_y_l93,O3,NO2,PM10,PM25
coord_x_l93,1.000000,-0.039298,-0.062872,0.069309,0.167238,0.067437
coord_y_l93,-0.039298,1.000000,-0.065233,0.000950,-0.018252,-0.017690
O3,-0.062872,-0.065233,1.000000,-0.354839,-0.186167,-0.216769
NO2,0.069309,0.000950,-0.354839,1.000000,0.544807,0.427693
PM10,0.167238,-0.018252,-0.186167,0.544807,1.000000,0.600095
PM25,0.067437,-0.017690,-0.216769,0.427693,0.600095,1.000000


In [ ]:
gg=newData.groupby(level=0)
sites=np.array([])
for n,g in gg:
    sites=np.append(sites,int(n))

In [ ]:
np.save('models/sites.npy',sites)

In [ ]:
data.isnull().sum()

In [23]:
newDataSet=None
groups=newData.groupby(level=0)
i=0
for name,g in groups:
    gg=g.reset_index().set_index('date').resample('D').mean().reset_index().set_index(['idPolair','date'])
    print(gg.shape)
    if(i==0):
        newDataSet=gg
    else:
        newDataSet=pd.concat([newDataSet,gg])
    i=i+1

(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1827, 6)


In [6]:
newDataSet=pd.read_csv('fillna/finalDays.csv')
newDataSet['date']=pd.to_datetime(newDataSet['date'])
newDataSet=newDataSet.set_index(['idPolair','date'])

In [25]:
newDataSet.corr()

,coord_x_l93,coord_y_l93,O3,NO2,PM10,PM25
coord_x_l93,1.000000,-0.039304,-0.075831,0.082735,0.191318,0.079886
coord_y_l93,-0.039304,1.000000,-0.078656,0.001152,-0.020882,-0.020922
O3,-0.075831,-0.078656,1.000000,-0.374434,-0.225111,-0.256703
NO2,0.082735,0.001152,-0.374434,1.000000,0.614957,0.489465
PM10,0.191318,-0.020882,-0.225111,0.614957,1.000000,0.641809
PM25,0.079886,-0.020922,-0.256703,0.489465,0.641809,1.000000


In [8]:
models=dict({})

In [26]:
groups=newDataSet.groupby(level=0)

In [31]:
model = Sequential()
model.add(LSTM(600,input_shape=(10,6),dropout=0.5))
model.add(Dense(12))
model.compile(loss='mean_squared_error', optimizer='adam')

In [32]:
for nb in range(50):
    print('__________________________________________')
    print(nb)
    print('__________________________________________')
    for name,x in groups:
        #K.clear_session()
        print(name)
        print('##########################')
        xx=x.iloc[:,2:]
        Xy=pd.concat([x,x.shift(-1),x.shift(-2),x.shift(-3),x.shift(-4),x.shift(-5),x.shift(-6),x.shift(-7),x.shift(-8),x.shift(-9),xx.shift(-10),xx.shift(-11),xx.shift(-12)],keys=['t','t+1','t+2','t+3','t+4','t+5','t+6','t+7','t+8','t+9','t+10','t+11','t+12'],axis=1).dropna()
        X=Xy.iloc[:,:60]
        y=Xy.iloc[:,60:]
        In=X.values.reshape(X.shape[0],10,6)
        Out=y.values.reshape(y.shape[0],12)
        model.fit(In,Out,verbose=2,epochs=10)
        model.reset_states()

__________________________________________
0
__________________________________________
7001
##########################
Epoch 1/10
 - 6s - loss: 826.4206
Epoch 2/10
 - 6s - loss: 408.1193
Epoch 3/10
 - 6s - loss: 265.0180
Epoch 4/10
 - 6s - loss: 221.4345
Epoch 5/10
 - 6s - loss: 197.8515
Epoch 6/10
 - 6s - loss: 177.6855
Epoch 7/10
 - 6s - loss: 172.0858
Epoch 8/10
 - 7s - loss: 162.9836
Epoch 9/10
 - 7s - loss: 160.0242
Epoch 10/10
 - 6s - loss: 154.2583
7004
##########################
Epoch 1/10
 - 6s - loss: 254.2574
Epoch 2/10
 - 7s - loss: 242.8149
Epoch 3/10
 - 6s - loss: 236.5776
Epoch 4/10
 - 7s - loss: 234.5280
Epoch 5/10
 - 6s - loss: 232.2235
Epoch 6/10
 - 6s - loss: 226.0337
Epoch 7/10
 - 6s - loss: 227.8301
Epoch 8/10
 - 7s - loss: 221.2491
Epoch 9/10
 - 6s - loss: 218.5582
Epoch 10/10
 - 6s - loss: 217.0327
7009
##########################
Epoch 1/10
 - 6s - loss: 202.1240
Epoch 2/10
 - 6s - loss: 199.2773
Epoch 3/10
 - 6s - loss: 197.1859
Epoch 4/10
 - 6s - loss: 194.687

Epoch 2/10
 - 6s - loss: 81.7770
Epoch 3/10
 - 6s - loss: 78.6588
Epoch 4/10
 - 6s - loss: 78.6995
Epoch 5/10
 - 6s - loss: 75.3067
Epoch 6/10
 - 6s - loss: 77.6113
Epoch 7/10
 - 6s - loss: 74.5364
Epoch 8/10
 - 6s - loss: 76.3625
Epoch 9/10
 - 6s - loss: 76.9636
Epoch 10/10
 - 6s - loss: 73.3731
7048
##########################
Epoch 1/10
 - 6s - loss: 140.5873
Epoch 2/10
 - 6s - loss: 116.7409
Epoch 3/10
 - 6s - loss: 109.0418
Epoch 4/10
 - 6s - loss: 104.1846
Epoch 5/10
 - 6s - loss: 98.7849
Epoch 6/10
 - 6s - loss: 99.0741
Epoch 7/10
 - 6s - loss: 99.1840
Epoch 8/10
 - 6s - loss: 92.1078
Epoch 9/10
 - 6s - loss: 95.9430
Epoch 10/10
 - 6s - loss: 89.9882
7049
##########################
Epoch 1/10
 - 6s - loss: 82.5059
Epoch 2/10
 - 6s - loss: 65.7564
Epoch 3/10
 - 6s - loss: 61.8735
Epoch 4/10
 - 6s - loss: 62.3704
Epoch 5/10
 - 6s - loss: 60.2322
Epoch 6/10
 - 6s - loss: 58.0520
Epoch 7/10
 - 6s - loss: 60.1210
Epoch 8/10
 - 6s - loss: 59.2017
Epoch 9/10
 - 7s - loss: 60.2042
Epoch 

Epoch 6/10
 - 6s - loss: 208.8252
Epoch 7/10
 - 6s - loss: 209.8473
Epoch 8/10
 - 6s - loss: 208.5623
Epoch 9/10
 - 6s - loss: 208.8464
Epoch 10/10
 - 6s - loss: 207.9691
20017
##########################
Epoch 1/10
 - 6s - loss: 246.8894
Epoch 2/10
 - 6s - loss: 182.2734
Epoch 3/10
 - 6s - loss: 180.5633
Epoch 4/10
 - 6s - loss: 174.2918
Epoch 5/10
 - 6s - loss: 173.8207
Epoch 6/10
 - 6s - loss: 173.4534
Epoch 7/10
 - 6s - loss: 175.1619
Epoch 8/10
 - 6s - loss: 172.8829
Epoch 9/10
 - 6s - loss: 171.1230
Epoch 10/10
 - 6s - loss: 175.9640
20019
##########################
Epoch 1/10
 - 6s - loss: 155.2518
Epoch 2/10
 - 6s - loss: 151.6034
Epoch 3/10
 - 6s - loss: 150.6942
Epoch 4/10
 - 6s - loss: 147.0101
Epoch 5/10
 - 6s - loss: 149.8825
Epoch 6/10
 - 6s - loss: 148.4469
Epoch 7/10
 - 6s - loss: 145.3209
Epoch 8/10
 - 6s - loss: 148.5946
Epoch 9/10
 - 6s - loss: 149.4132
Epoch 10/10
 - 6s - loss: 146.8032
20029
##########################
Epoch 1/10
 - 6s - loss: 77.9781
Epoch 2/10
 - 6

 - 7s - loss: 123.7336
Epoch 7/10
 - 6s - loss: 125.4081
Epoch 8/10
 - 6s - loss: 125.6996
Epoch 9/10
 - 6s - loss: 124.6043
Epoch 10/10
 - 6s - loss: 123.1553
27010
##########################
Epoch 1/10
 - 6s - loss: 152.4632
Epoch 2/10
 - 7s - loss: 74.4177
Epoch 3/10
 - 6s - loss: 72.7453
Epoch 4/10
 - 6s - loss: 72.3768
Epoch 5/10
 - 7s - loss: 69.8558
Epoch 6/10
 - 6s - loss: 67.7193
Epoch 7/10
 - 6s - loss: 66.5948
Epoch 8/10
 - 6s - loss: 67.9178
Epoch 9/10
 - 6s - loss: 66.7041
Epoch 10/10
 - 6s - loss: 66.8351
29421
##########################
Epoch 1/10
 - 6s - loss: 162.9547
Epoch 2/10
 - 6s - loss: 147.3711
Epoch 3/10
 - 6s - loss: 148.5373
Epoch 4/10
 - 6s - loss: 144.0282
Epoch 5/10
 - 6s - loss: 139.8400
Epoch 6/10
 - 6s - loss: 139.7140
Epoch 7/10
 - 6s - loss: 138.9281
Epoch 8/10
 - 6s - loss: 141.2021
Epoch 9/10
 - 6s - loss: 140.3998
Epoch 10/10
 - 6s - loss: 137.4533
29423
##########################
Epoch 1/10
 - 6s - loss: 114.6340
Epoch 2/10
 - 6s - loss: 117.7943


Epoch 9/10
 - 6s - loss: 114.8374
Epoch 10/10
 - 6s - loss: 114.2464
33212
##########################
Epoch 1/10
 - 6s - loss: 131.9409
Epoch 2/10
 - 6s - loss: 119.0055
Epoch 3/10
 - 6s - loss: 117.6120
Epoch 4/10
 - 6s - loss: 115.3469
Epoch 5/10
 - 6s - loss: 117.3714
Epoch 6/10
 - 6s - loss: 116.5992
Epoch 7/10
 - 6s - loss: 115.6177
Epoch 8/10
 - 6s - loss: 117.0140
Epoch 9/10
 - 6s - loss: 115.4172
Epoch 10/10
 - 6s - loss: 114.2305
33213
##########################
Epoch 1/10
 - 6s - loss: 25.3409
Epoch 2/10
 - 6s - loss: 22.8628
Epoch 3/10
 - 6s - loss: 21.2697
Epoch 4/10
 - 7s - loss: 20.8224
Epoch 5/10
 - 6s - loss: 21.5459
Epoch 6/10
 - 6s - loss: 20.8504
Epoch 7/10
 - 7s - loss: 21.2237
Epoch 8/10
 - 6s - loss: 21.4408
Epoch 9/10
 - 6s - loss: 19.5499
Epoch 10/10
 - 7s - loss: 19.0174
33220
##########################
Epoch 1/10
 - 6s - loss: 173.6955
Epoch 2/10
 - 6s - loss: 156.2538
Epoch 3/10
 - 7s - loss: 156.2362
Epoch 4/10
 - 6s - loss: 153.0938
Epoch 5/10
 - 6s - loss:

...

Epoch 8/10
 - 6s - loss: 113.3763
Epoch 9/10
 - 6s - loss: 115.1090
Epoch 10/10
 - 6s - loss: 112.0117
15031
##########################
Epoch 1/10
 - 6s - loss: 105.9615
Epoch 2/10
 - 6s - loss: 69.4190
Epoch 3/10
 - 6s - loss: 61.8892
Epoch 4/10
 - 6s - loss: 58.7870
Epoch 5/10
 - 7s - loss: 60.0285
Epoch 6/10
 - 6s - loss: 57.6284
Epoch 7/10
 - 6s - loss: 58.0175
Epoch 8/10
 - 6s - loss: 54.8071
Epoch 9/10
 - 6s - loss: 55.8058
Epoch 10/10
 - 6s - loss: 55.6494
15038
##########################
Epoch 1/10
 - 6s - loss: 197.4161
Epoch 2/10
 - 6s - loss: 139.0281
Epoch 3/10
 - 6s - loss: 132.1956
Epoch 4/10
 - 6s - loss: 126.8795
Epoch 5/10
 - 6s - loss: 126.0626
Epoch 6/10
 - 6s - loss: 126.2213
Epoch 7/10
 - 6s - loss: 127.0605
Epoch 8/10
 - 7s - loss: 125.4703
Epoch 9/10
 - 6s - loss: 126.8055
Epoch 10/10
 - 6s - loss: 127.5870
15039
##########################
Epoch 1/10
 - 6s - loss: 125.8976
Epoch 2/10
 - 6s - loss: 112.0249
Epoch 3/10
 - 6s - loss: 107.9724
Epoch 4/10
 - 7s - loss

Epoch 9/10
 - 6s - loss: 177.3211
Epoch 10/10
 - 6s - loss: 171.0818
20063
##########################
Epoch 1/10
 - 6s - loss: 178.2175
Epoch 2/10
 - 6s - loss: 157.4271
Epoch 3/10
 - 6s - loss: 146.7753
Epoch 4/10
 - 6s - loss: 142.9984
Epoch 5/10
 - 6s - loss: 136.2063
Epoch 6/10
 - 6s - loss: 133.9773
Epoch 7/10
 - 7s - loss: 133.8733
Epoch 8/10
 - 6s - loss: 130.7435
Epoch 9/10
 - 6s - loss: 131.1478
Epoch 10/10
 - 6s - loss: 133.5443
20065
##########################
Epoch 1/10
 - 6s - loss: 225.5848
Epoch 2/10
 - 6s - loss: 219.1154
Epoch 3/10
 - 6s - loss: 218.3898
Epoch 4/10
 - 6s - loss: 220.6568
Epoch 5/10
 - 6s - loss: 215.7511
Epoch 6/10
 - 6s - loss: 221.5712
Epoch 7/10
 - 6s - loss: 214.7883
Epoch 8/10
 - 6s - loss: 214.2377
Epoch 9/10
 - 6s - loss: 212.0486
Epoch 10/10
 - 6s - loss: 214.4166
20069
##########################
Epoch 1/10
 - 6s - loss: 192.1589
Epoch 2/10
 - 6s - loss: 170.2050
Epoch 3/10
 - 6s - loss: 165.8696
Epoch 4/10
 - 6s - loss: 154.9750
Epoch 5/10
 - 

Epoch 10/10
 - 6s - loss: 106.5734
33105
##########################
Epoch 1/10
 - 6s - loss: 45.9388
Epoch 2/10
 - 7s - loss: 42.5847
Epoch 3/10
 - 7s - loss: 42.3725
Epoch 4/10
 - 6s - loss: 40.7627
Epoch 5/10
 - 6s - loss: 40.8789
Epoch 6/10
 - 6s - loss: 39.1546
Epoch 7/10
 - 6s - loss: 38.6413
Epoch 8/10
 - 6s - loss: 37.0939
Epoch 9/10
 - 6s - loss: 38.1569
Epoch 10/10
 - 6s - loss: 37.9724
33111
##########################
Epoch 1/10
 - 6s - loss: 80.2723
Epoch 2/10
 - 6s - loss: 79.3635
Epoch 3/10
 - 6s - loss: 80.3281
Epoch 4/10
 - 6s - loss: 81.4494
Epoch 5/10
 - 6s - loss: 80.3832
Epoch 6/10
 - 6s - loss: 81.1627
Epoch 7/10
 - 6s - loss: 80.0140
Epoch 8/10
 - 6s - loss: 80.1601
Epoch 9/10
 - 6s - loss: 77.0320
Epoch 10/10
 - 6s - loss: 78.6332
33114
##########################
Epoch 1/10
 - 6s - loss: 97.5173
Epoch 2/10
 - 6s - loss: 89.0476
Epoch 3/10
 - 6s - loss: 89.9615
Epoch 4/10
 - 6s - loss: 89.8808
Epoch 5/10
 - 6s - loss: 89.6752
Epoch 6/10
 - 6s - loss: 86.8810
Epoch 

 - 6s - loss: 119.9057
Epoch 3/10
 - 6s - loss: 122.5507
Epoch 4/10
 - 6s - loss: 118.1962
Epoch 5/10
 - 6s - loss: 118.5683
Epoch 6/10
 - 6s - loss: 116.5632
Epoch 7/10
 - 7s - loss: 116.5396
Epoch 8/10
 - 6s - loss: 119.4967
Epoch 9/10
 - 6s - loss: 117.8776
Epoch 10/10
 - 7s - loss: 118.7213
36002
##########################
Epoch 1/10
 - 6s - loss: 128.7666
Epoch 2/10
 - 6s - loss: 122.1872
Epoch 3/10
 - 6s - loss: 118.3679
Epoch 4/10
 - 6s - loss: 116.4484
Epoch 5/10
 - 6s - loss: 115.7269
Epoch 6/10
 - 6s - loss: 116.8750
Epoch 7/10
 - 6s - loss: 116.8521
Epoch 8/10
 - 6s - loss: 115.0138
Epoch 9/10
 - 6s - loss: 111.8843
Epoch 10/10
 - 7s - loss: 111.9731
36003
##########################
Epoch 1/10
 - 6s - loss: 133.7032
Epoch 2/10
 - 6s - loss: 109.1162
Epoch 3/10
 - 6s - loss: 106.0147
Epoch 4/10
 - 6s - loss: 105.0227
Epoch 5/10
 - 6s - loss: 104.6284
Epoch 6/10
 - 6s - loss: 104.2380
Epoch 7/10
 - 6s - loss: 103.1164
Epoch 8/10
 - 6s - loss: 103.2704
Epoch 9/10
 - 6s - loss: 

KeyboardInterrupt: 

In [33]:
model.save('lklMarraWe7di.model')

In [ ]:

for name,x in groups:
    Xy=pd.concat([x,x.shift(-1),x.shift(-2),x.shift(-3),x.shift(-4),x.shift(-5),x.shift(-6),x.shift(-7),x.shift(-8),x.shift(-9),x.shift(-10),x.shift(-11),x.shift(-12)],keys=['t','t+1','t+2','t+3','t+4','t+5','t+6','t+7','t+8','t+9','t+10','t+11','t+12'],axis=1).dropna()
    X=Xy.iloc[:,:40]
    y=Xy.iloc[:,40:]
    In=X.values.reshape(X.shape[0],10,4)
    Out=y.values.reshape(y.shape[0],12)
    model = models[name]=model
    model.reset_states()
    model.fit(In,Out,verbose=2,epochs=100)
    models[name]=model
    break

In [34]:
model

In [ ]:
newDataSet.shift(-1).head()

In [ ]:
import time

In [ ]:
x=time.time()

In [ ]:
y=time.time()
y-x

In [ ]:
def load_models(dir='models/'):
    x=time.time()
    sites=np.load(dir+'sites.npy')
    models=dict({})
    for i in sites:
        print(i)
        models[i]=load_model(dir+str(int(i))+'.model')
    y=time.time()
    print((y-x))
    return models

In [ ]:
models=load_models()

In [ ]:
models

In [ ]:
K.clear_session()

In [ ]:
x=time.time()
m=load_model('models/33111.model')
y=time.time()

In [ ]:
x1=time.time()
m1=load_model('models/33111.model',compile=False,)
y1=time.time()

In [ ]:
y-x

In [ ]:
y1-x1

In [ ]:
m.save_weights('models/w.test')

In [ ]:
m1.weights

In [ ]:
mo = Sequential()
mo.add(LSTM(200,input_shape=(4,10),dropout=0.4))
mo.add(Dense(12))
mo.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
x2=time.time()
mo.load_weights('models/w.test')
y2=time.time()

In [ ]:
y2-x2

In [ ]:
newDataSet=pd.read_csv('fillna/finalDays.csv')

In [ ]:
newDataSet.set_index(['idPolair','date'],inplace=True)

In [ ]:
groups=newDataSet.groupby(level=0)

In [ ]:
mmm=dict({})

for name,x in groups:
    Xy=pd.concat([x,x.shift(-1),x.shift(-2),x.shift(-3),x.shift(-4),x.shift(-5),x.shift(-6),x.shift(-7),x.shift(-8),x.shift(-9),x.shift(-10),x.shift(-11),x.shift(-12)],keys=['t','t+1','t+2','t+3','t+4','t+5','t+6','t+7','t+8','t+9','t+10','t+11','t+12'],axis=1).dropna()
    X=Xy.iloc[:,:40]
    y=Xy.iloc[:,40:]
    In=X.values.reshape(X.shape[0],4,10)
    Out=y.values.reshape(y.shape[0],12)
    model = Sequential()
    model.add(LSTM(20,input_shape=(4,10),dropout=0.4))
    model.add(Dense(12))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(In,Out,verbose=2,epochs=100)
    mmm[name]=model
    mmm[name].save('models2/'+str(int(name))+'.model')

In [ ]:
x2=time.time()
m2=load_model('models2/7001.model')
y2=time.time()

In [ ]:
y2-x2

In [ ]:
x2=time.time()
m2=load_model('models2/7001.model',compile=False)
y2=time.time()

In [ ]:
y2-x2

In [ ]:
import h5py

In [ ]:
def load_models2(dir='models/'):
    x=time.time()
    sites=np.load(dir+'sites.npy')
    models=dict({})
    for i in sites:
        K.clear_session()
        print(i)
        models[i]=load_model(dir+str(int(i))+'.model')
    y=time.time()
    print((y-x))
    return models

In [ ]:
momo=load_models2()

In [42]:
x=utils.load_data()

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


In [43]:
x.keys()

dict_keys(['sites', 'chimeres', 'geops', 'meteo', 'concentrations'])

In [ ]:
y=utils.get_data_day(10,x)

In [ ]:
gg=y[0]['NO2'].set_index(['idPolair','date']).sort_index().groupby(level=0)

In [ ]:
for i,j in gg:
    print(j)
    break

In [74]:
def run_predict(year=2016, max_days=10, dirname="../Data/training", list_days=None):
    """
    year : year to be evaluated
    max_days: number of past days allowed to predict a given day (set to 10 on the platform)
    dirname: path to the dataset
    list_days: list of days to be evaluated (if None the full year is evaluated)
    """

    overall_start = time.time()  # <== Mark starting time
    data = load_data(year=year, dirname=dirname)  # load all data files
    sites = data["sites"]  # get sites info
    day_results = dict({})
    if list_days is None:
        if calendar.isleap(year):  # check if year is leap
            list_days = range(366)
        else:
            list_days = range(365)
    for day in list_days:
        print(day)
        return  get_data_day(day, data, max_days=max_days,
                                                                              year=year)  # you will get an extraction of the year datasets, limited to the past max_days for each day
        day_results[day] = predict(day, sites, chimeres_day, geops_day, meteo_day,
                                   concentrations_day)  # do the prediction

    overall_time_spent = time.time() - overall_start  # end computation time
    pickle.dump(day_results, open('submission/results.pk', 'wb'))  # save results
    pickle.dump(overall_time_spent, open('submission/time.pk', 'wb'))  # save computation time

In [75]:
chimeres_day, geops_day, meteo_day, concentrations_day =run_predict(list_days=[14])

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


14


In [ ]:
c=concentrations_day

In [ ]:
c.keys()

In [84]:
def toPredcit(concentrations_day,sites):
    sites['idPolair']=pd.to_numeric(sites['idPolair'])
    sites=sites.set_index('idPolair').loc[:,['coord_x_l93','coord_y_l93']]/10000
    O3=pd.DataFrame(concentrations_day['O3'].set_index(['idPolair','date'])['Valeur'])
    O3.columns=['O3']
    NO2=pd.DataFrame(concentrations_day['NO2'].set_index(['idPolair','date'])['Valeur'])
    NO2.columns=['NO2']
    PM10=pd.DataFrame(concentrations_day['PM10'].set_index(['idPolair','date'])['Valeur'])
    PM10.columns=['PM10']
    PM25=pd.DataFrame(concentrations_day['PM25'].set_index(['idPolair','date'])['Valeur'])
    PM25.columns=['PM25']
    cons=pd.merge(O3,pd.merge(NO2,pd.merge(PM10,PM25,left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer').reset_index()
    cons['idPolair']=pd.to_numeric(cons['idPolair'])
    cons['date']=pd.to_datetime(cons['date'])
    cons=cons.set_index('idPolair')
    X=pd.merge(sites,cons,left_index=True,right_index=True)
    X=X.reset_index()
    X=X.set_index(['idPolair','date']).sort_index()
    i=0

    means=X.reset_index().set_index('date').resample('D').mean()
    del means['idPolair']
    meanOfmeans=pd.DataFrame((np.array(means.mean().tolist()).reshape(1,6)),columns=['X','Y','O3', 'NO2', 'PM10', 'PM25'])
    meanOfmeans=pd.concat([meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans],axis=1)
    newX=None
    groups=X.groupby(level=0)
    i=0
    for name,g in groups:
        gg=g.reset_index().set_index('date').resample('D').mean().reset_index().set_index(['idPolair','date']).fillna(method='ffill').fillna(method='bfill').fillna(means).fillna(0)
        shape=gg.shape[0]
        gg=pd.DataFrame(np.array(pd.concat([gg.shift(10),gg.shift(9),gg.shift(8),gg.shift(7),gg.shift(6),gg.shift(5),gg.shift(4),gg.shift(3),gg.shift(2),gg.shift(1)],axis=1).iloc[shape-1,:].tolist()).reshape(1,60),index=[name],columns=['X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25'])
        meanOfmeans.index=[name]
        gg=gg.fillna(meanOfmeans)
        if(i==0):
            newX=gg
        else:
            newX=pd.concat([newX,gg])
        i=i+1
    return newX

In [85]:
newX=toPredcit(concentrations_day,x['sites'])

In [86]:
newX

,X,Y,O3,NO2,PM10,PM25,X,Y,O3,NO2,...,O3,NO2,PM10,PM25,X,Y,O3,NO2,PM10,PM25
7001,70.366789,651.834092,76.958333,3.708333,11.620183,7.926931,70.366789,651.834092,55.416667,8.500000,...,63.000000,6.375000,15.852232,8.367089,70.366789,651.834092,45.000000,15.916667,17.225420,14.436170
7004,70.884203,652.196805,67.583333,10.916667,4.750000,4.000000,70.884203,652.196805,47.375000,18.416667,...,51.000000,19.958333,12.000000,5.000000,70.884203,652.196805,32.416667,27.500000,10.045455,5.000000
7009,70.679922,651.917674,68.916667,12.583333,5.083333,7.926931,70.679922,651.917674,50.166667,18.666667,...,51.416667,22.250000,12.666667,8.367089,70.679922,651.917674,35.041667,32.125000,10.208333,14.436170
7010,66.909266,658.272251,43.671194,14.916667,8.666667,7.926931,66.909266,658.272251,32.159854,20.625000,...,44.970440,25.750000,16.875000,8.367089,66.909266,658.272251,28.386628,29.291667,12.625000,14.436170
7011,65.554787,642.505177,43.671194,30.254881,11.620183,7.926931,65.554787,642.505177,32.159854,32.746531,...,44.970440,33.372130,15.852232,8.367089,65.554787,642.505177,28.386628,41.351121,17.225420,14.436170
7012,76.949688,643.877414,43.671194,30.254881,11.620183,7.926931,76.949688,643.877414,32.159854,32.746531,...,44.970440,33.372130,15.852232,8.367089,76.949688,643.877414,28.386628,41.351121,17.225420,14.436170
7013,66.958584,658.231669,65.333333,8.875000,11.620183,7.926931,66.958584,658.231669,48.541667,12.958333,...,47.250000,17.041667,15.852232,8.367089,66.958584,658.231669,32.166667,21.333333,17.225420,14.436170
7014,65.422892,642.229442,43.671194,30.254881,11.620183,7.926931,65.422892,642.229442,32.159854,32.746531,...,44.970440,33.372130,15.852232,8.367089,65.422892,642.229442,28.386628,41.351121,17.225420,14.436170
7015,69.724646,651.916641,80.666667,30.254881,11.620183,7.926931,69.724646,651.916641,69.083333,32.746531,...,75.291667,33.372130,15.852232,8.367089,69.724646,651.916641,73.833333,41.351121,17.225420,14.436170
7016,66.932515,658.054734,66.250000,30.254881,11.620183,7.926931,66.932515,658.054734,50.833333,32.746531,...,50.375000,33.372130,15.852232,8.367089,66.932515,658.054734,35.250000,41.351121,17.225420,14.436170


In [44]:
newX.iloc[:5,7*4:]

,O3,NO2,PM10,PM25,O3,NO2,PM10,PM25,O3,NO2,PM10,PM25
7001,78.041667,1.291667,7.948734,4.494505,63.000000,6.375000,15.852232,8.367089,45.000000,15.916667,17.225420,14.43617
7004,68.041667,8.166667,3.791667,5.000000,51.000000,19.958333,12.000000,5.000000,32.416667,27.500000,10.045455,5.00000
7009,72.125000,7.708333,4.500000,4.494505,51.416667,22.250000,12.666667,8.367089,35.041667,32.125000,10.208333,14.43617
7010,55.291966,16.727273,11.916667,4.494505,44.970440,25.750000,16.875000,8.367089,28.386628,29.291667,12.625000,14.43617
7011,55.291966,23.875803,7.948734,4.494505,44.970440,33.372130,15.852232,8.367089,28.386628,41.351121,17.225420,14.43617


In [47]:
yy['NO2']

{'7001': {'D0': array([12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726,
         12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726,
         12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726,
         12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726],
        dtype=float32),
  'D1': array([16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081,
         16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081,
         16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081,
         16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081],
        dtype=float32),
  'D2': array([17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357,
         17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357,
         17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357,
         17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357],
        dtype=float32)}

In [91]:
model_lkl=load_model('lklMarraWe7di.model')

def predict(X,model,dir='models/'):
    
    #define a dictionary to return the result
    results = dict({})
    
    #add the 4 pol to the dict
    for pol in ["PM10", "PM25", "O3", "NO2"]:
        results[pol] = dict({})
        #add each idPolair to each pol in the dict
        sites=np.load('models/sites.npy')
        for idPolair in sites:
            results[pol][''+str(int(idPolair))] = dict({})
        
    
    #for each pol predict and add the result
    for index,row in X.iterrows():
        
        
        #predict the 12 values
        y_predict = model.predict(row.values.reshape(1,10,6))
        
        #assign each value to the right place
        idPolair=str(int(index))
        print(idPolair)
        results["NO2"][idPolair]["D0"] = np.full((24), y_predict[0, 1])
        results["NO2"][idPolair]["D1"] = np.full((24),y_predict[0, 5])
        results["NO2"][idPolair]["D2"] = np.full((24),y_predict[0, 9])
        results["O3"][idPolair]["D0"] = np.full((24),y_predict[0, 0])
        results["O3"][idPolair]["D1"] = np.full((24),y_predict[0, 4])
        results["O3"][idPolair]["D2"] = np.full((24),y_predict[0, 8])
        results["PM10"][idPolair]["D0"] = np.full((24),y_predict[0, 2])
        results["PM10"][idPolair]["D1"] = np.full((24),y_predict[0, 6])
        results["PM10"][idPolair]["D2"] = np.full((24),y_predict[0, 10])
        results["PM25"][idPolair]["D0"] = np.full((24),y_predict[0, 3])
        results["PM25"][idPolair]["D1"] = np.full((24),y_predict[0, 7])
        results["PM25"][idPolair]["D2"] = np.full((24),y_predict[0, 11])
        
    return results
        
        
        
        

In [92]:
yy=predict(newX,model_lkl)

7001
7004
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7020
7022
7028
7029
7031
7032
7034
7039
7042
7043
7045
7048
7049
7051
7052
7053
7054
7056
7057
7058
15013
15017
15018
15031
15038
15039
15043
15045
15046
15048
15049
20004
20013
20017
20019
20029
20031
20037
20045
20046
20047
20048
20049
20061
20062
20063
20065
20069
20070
27002
27003
27004
27005
27007
27008
27010
29421
29423
29424
29426
29428
29429
29439
29440
29441
33101
33102
33105
33111
33114
33120
33121
33122
33201
33202
33203
33211
33212
33213
33220
33232
33233
33235
33302
33303
33305
33367
33374
33414
36001
36002
36003
36005
36019
36021


In [ ]:



for idPolair in sites.idPolair:
    if int(idPolair) in X:
        y_predict = model.predict(np.array([X[int(idPolair)]]))
        results["NO2"][idPolair]["D0"] = np.full((24), y_predict[0, 0])
        results["NO2"][idPolair]["D1"] = np.full((24),y_predict[0, 4])
        results["NO2"][idPolair]["D2"] = np.full((24),y_predict[0, 8])
        results["O3"][idPolair]["D0"] = np.full((24),y_predict[0, 1])
        results["O3"][idPolair]["D1"] = np.full((24),y_predict[0, 5])
        results["O3"][idPolair]["D2"] = np.full((24),y_predict[0, 9])
        results["PM10"][idPolair]["D0"] = np.full((24),y_predict[0, 2])
        results["PM10"][idPolair]["D1"] = np.full((24),y_predict[0, 6])
        results["PM10"][idPolair]["D2"] = np.full((24),y_predict[0, 10])
        results["PM25"][idPolair]["D0"] = np.full((24),y_predict[0, 3])
        results["PM25"][idPolair]["D1"] = np.full((24),y_predict[0, 7])
        results["PM25"][idPolair]["D2"] = np.full((24),y_predict[0, 11])
    else:

In [93]:
yy

{'PM10': {'7001': {'D0': array([8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937,
          8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937,
          8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937,
          8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937],
         dtype=float32),
   'D1': array([8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651,
          8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651,
          8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651,
          8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651],
         dtype=float32),
   'D2': array([8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294,
          8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294,
          8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294,
          8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294],
         dtype=float32)},
  '7004': {'D0': array([10.453365, 10.453365, 1

In [37]:
sites.loc[:,['coord_x_l93','coord_y_l93']]

,coord_x_l93,coord_y_l93
idPolair,,
15013,91.466844,644.596850
15017,91.097088,645.816550
15018,90.269900,647.672600
15031,97.331581,643.921300
15038,91.620200,645.751600
15039,91.241138,645.465050
15043,91.491600,645.510200
15045,91.067863,644.343650
15046,91.365000,645.717500


In [98]:
import p

In [99]:
p.run_predict()

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


0
2.169496536254883
1
2.0750174522399902
2
2.1517012119293213
3
2.4328041076660156
4
2.3769967555999756
5
2.6255435943603516
6
2.7225472927093506
7
3.2900400161743164
8
3.1982760429382324
9
3.336198568344116
10
3.463796615600586
11
3.0654845237731934
12
3.8391804695129395
13
3.255401134490967
14
3.319997787475586
15
3.0552196502685547
16
3.3727381229400635
17
3.9056129455566406
18
3.833986282348633
19
3.608930826187134
20
3.921595335006714
21
3.736147880554199
22
3.403850555419922
23
3.0764482021331787
24
3.6064016819000244
25
3.739269256591797
26
3.5093483924865723
27
3.1874783039093018
28
3.9500675201416016
29
4.05430793762207
30
4.233757257461548
31
3.9810845851898193
32
3.5930826663970947
33
3.0700342655181885
34
3.733464002609253
35
3.0650670528411865
36
3.278904438018799
37
3.073699712753296
38
3.8197226524353027
39
3.4166862964630127
40
3.414902925491333
41
3.101870059967041
42
3.2647945880889893
43
3.086899995803833
44
3.809231996536255
45
3.3476297855377197
46
3.50440907478332

In [103]:
import scoring

In [102]:
del scoring

In [105]:
model.reset_states()

In [107]:
from test import pr

ModuleNotFoundError: No module named 'test.prediction'; 'test' is not a package

In [ ]:
from utils import load_data, get_data_day
import numpy as np
import pandas as pd

import pickle
import datetime
import calendar
import time
import os

import utils
import logging
import pickle


logging.basicConfig(level=logging.INFO)

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.models import load_model
from keras import backend as K



dirname = os.path.dirname(__file__)

"""

Author: Hussein Awala

Models: Keras LSTM for each site taking the concentrations for the previous 10 days as input and predict the concentration for the 3 next days.

"""



def predict(day, sites, chimeres_day, geops_day, meteo_day, concentrations_day, model=None):

    start=time.time()

    X=toPredcit(concentrations_day)
    
    #define a dictionary to return the result
    results = dict({})
    
    #add the 4 pol to the dict
    for pol in ["PM10", "PM25", "O3", "NO2"]:
        results[pol] = dict({})
        #add each idPolair to each pol in the dict
        #sites=np.load('models/sites.npy')
        for idPolair in sites.idPolair:
            results[pol][''+str(int(idPolair))] = dict({})
        
    #results[pol]['33374'] = dict({})
    
    #for each pol predict and add the result
    for index,row in X.iterrows():
        
        #remove old model from GPU to speed up the model
        K.clear_session()
        
        #load the model of this site
        filename = os.path.join(dirname,'models/'+ str(int(index))+'.model')
        model=load_model(filename)
        
        #predict the 12 values
        y_predict = model.predict(row.values.reshape(1,10,4))
        
        #assign each value to the right place
        idPolair=str(int(index))
        results["NO2"][idPolair]["D0"] = np.full((24), y_predict[0, 1])
        results["NO2"][idPolair]["D1"] = np.full((24),y_predict[0, 5])
        results["NO2"][idPolair]["D2"] = np.full((24),y_predict[0, 9])
        results["O3"][idPolair]["D0"] = np.full((24),y_predict[0, 0])
        results["O3"][idPolair]["D1"] = np.full((24),y_predict[0, 4])
        results["O3"][idPolair]["D2"] = np.full((24),y_predict[0, 8])
        results["PM10"][idPolair]["D0"] = np.full((24),y_predict[0, 2])
        results["PM10"][idPolair]["D1"] = np.full((24),y_predict[0, 6])
        results["PM10"][idPolair]["D2"] = np.full((24),y_predict[0, 10])
        results["PM25"][idPolair]["D0"] = np.full((24),y_predict[0, 3])
        results["PM25"][idPolair]["D1"] = np.full((24),y_predict[0, 7])
        results["PM25"][idPolair]["D2"] = np.full((24),y_predict[0, 11])
    end=time.time()
    print((end-start))
    return results
        
        
        
        


def run_predict(year=2016, max_days=10, dirname="../Data/training", list_days=None):
    """
    year : year to be evaluated
    max_days: number of past days allowed to predict a given day (set to 10 on the platform)
    dirname: path to the dataset
    list_days: list of days to be evaluated (if None the full year is evaluated)
    """

    overall_start = time.time()  # <== Mark starting time
    data = load_data(year=year, dirname=dirname)  # load all data files
    sites = data["sites"]  # get sites info
    day_results = dict({})
    if list_days is None:
        if calendar.isleap(year):  # check if year is leap
            list_days = range(366)
        else:
            list_days = range(365)
    for day in list_days:
        print(day)
        chimeres_day, geops_day, meteo_day, concentrations_day = get_data_day(day, data, max_days=max_days,
                                                                              year=year)  # you will get an extraction of the year datasets, limited to the past max_days for each day
        day_results[day] = predict(day, sites, chimeres_day, geops_day, meteo_day,
                                   concentrations_day)  # do the prediction

    overall_time_spent = time.time() - overall_start  # end computation time
    pickle.dump(day_results, open('submission/results.pk', 'wb'))  # save results
    pickle.dump(overall_time_spent, open('submission/time.pk', 'wb'))  # save computation time



def toPredcit(concentrations_day):
    O3=pd.DataFrame(concentrations_day['O3'].set_index(['idPolair','date'])['Valeur'])
    O3.columns=['O3']
    NO2=pd.DataFrame(concentrations_day['NO2'].set_index(['idPolair','date'])['Valeur'])
    NO2.columns=['NO2']
    PM10=pd.DataFrame(concentrations_day['PM10'].set_index(['idPolair','date'])['Valeur'])
    PM10.columns=['PM10']
    PM25=pd.DataFrame(concentrations_day['PM25'].set_index(['idPolair','date'])['Valeur'])
    PM25.columns=['PM25']
    X=pd.merge(O3,pd.merge(NO2,pd.merge(PM10,PM25,left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer')
    X=X.reset_index()
    X['idPolair']=pd.to_numeric(X['idPolair'])
    X['date']=pd.to_datetime(X['date'])
    X=X.set_index(['idPolair','date']).sort_index()
    i=0


    means=X.reset_index().set_index('date').resample('D').mean()
    del means['idPolair']
    meanOfmeans=pd.DataFrame((np.array(means.mean().tolist()).reshape(1,4)),columns=['O3', 'NO2', 'PM10', 'PM25'])
    meanOfmeans=pd.concat([meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans],axis=1)
    newX=None
    groups=X.groupby(level=0)
    i=0
    for name,g in groups:
        gg=g.reset_index().set_index('date').resample('D').mean().reset_index().set_index(['idPolair','date']).fillna(method='ffill').fillna(method='bfill').fillna(means).fillna(0)
        shape=gg.shape[0]
        gg=pd.DataFrame(np.array(pd.concat([gg.shift(10),gg.shift(9),gg.shift(8),gg.shift(7),gg.shift(6),gg.shift(5),gg.shift(4),gg.shift(3),gg.shift(2),gg.shift(1)],axis=1).iloc[shape-1,:].tolist()).reshape(1,40),index=[name],columns=['O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25'])
        meanOfmeans.index=[name]
        gg=gg.fillna(meanOfmeans)
        if(i==0):
            newX=gg
        else:
            newX=pd.concat([newX,gg])
        i=i+1
    return newX